In [1]:
import pandas as pd
import regex as re
from collections import Counter

import json
import time
import requests
import collections
import matplotlib.pyplot as plt
import numpy as np
import pprint
import pycountry 
import pycountry_convert as pc
from wipo_ipc import Ipc

In [64]:
# so some WO patents cannot be identified --> if adding 1 to 'A' endings, it can be improved, but still doesn't work for all of them. 
# some KR patents cannot be found either
# URLs for these are also broken
# proceed for now with those that I can identify without editing

## 1. Get patent data

In [2]:
# open data
df = pd.read_csv(R'X:\5_Research\Paul\data\genes_of_technology\MABPAT_patent_nums_141123_COMPLETED.csv', index_col=0).reset_index().drop(columns=['index'])#.rename(columns={0:'patentID'})

In [58]:
df.head()

,organism,patent_num,patent_url,mod_patent_num
0,Vibrio cholerae,WO2004024931A2,https://patents.google.com/patent/WO2004024931...,WO2004024931A2
1,Vibrio cholerae,JP2010213712A,https://patents.google.com/patent/JP2010213712...,JP2010213712A
2,Vibrio cholerae,JP2012503992A,https://patents.google.com/patent/JP2012503992...,JP2012503992A
3,Vibrio cholerae,JP2012503993A,https://patents.google.com/patent/JP2012503993...,JP2012503993A
4,Vibrio cholerae,JP2011062202A,https://patents.google.com/patent/JP2011062202...,JP2011062202A


In [65]:
modified_patent_ids = []

for item in list(df['patent_num']):
    if item.startswith('WO') and not item[-1].isdigit():
        modified_patent_ids.append(item + '1')
    else:
        modified_patent_ids.append(item)

df['mod_patent_num'] = modified_patent_ids

In [66]:
# make list of patentIDs (got to include the double quotes); delete duplicates
data_list = json.dumps(list(set(df['patent_num'])))

len(list(set(df['patent_num'])))


4779

In [90]:
data_list

'["US6900303A", "KR1020200061374A", "WO2020241486A", "JP2018121648A", "WO2006112040A", "WO2005069724A2", "JP2012213382A", "JP2013000064A", "US6692917A", "US9688978B2", "JP2015211679A", "EP2865750A2", "US9783820B2", "US7358042A", "JP2012520070A", "US7807392B1", "JP2006149307A", "US7723498B2", "US9134323B2", "JP2002360259A", "EP3517962A1", "JP2017209118A", "JP2002142776A", "WO2009120731A2", "WO2008074891A2", "EP3372682A1", "WO2009134276A1", "US7531320A", "US8937046B2", "WO9821327A1", "JP2009504146A", "WO2020123716A1", "JP2018534943A", "US7175658A", "JP2009132704A", "US8716462B2", "EP2408795A1", "WO9931260A2", "JP2015516800A", "JP2010505418A", "US6410715A", "WO2018128973A1", "WO2019203802A1", "JP2017093325A", "KR1020120116612A", "EP1961765A1", "US9428761B2", "US9062118B2", "JP1999035493A", "US6838259A", "EP3524676A1", "KR1020170140801A", "EP2363492A2", "EP2855699A1", "KR1020200048066A", "JP2014198695A", "WO2007136762A", "WO2004065415A2", "US9409987B2", "EP3589373A1", "JP2018501801A", "JP2

In [1]:
#Load API key
my_key = open(R"C:\\Users\\pauld\\OneDrive\\PhD\\Code\\keys\\thelens.txt", "r").read()

In [57]:
#find error: 
import requests
url = 'https://api.lens.org/patent/search'
data_x = '''{
              "query": {
                  "terms":  {
                      "ids": ["WO2007110314A1"]
                  }
              },
              "include": ["biblio", "doc_key"]
}'''
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}
response = requests.post(url, data=data_x, headers=headers)
if response.status_code != requests.codes.ok:
      print(response.status_code)
else:
      print(response.text)

{"total":0,"max_score":0.0,"data":[]}


In [67]:
data = []
url = 'https://api.lens.org/patent/search'

# include fields

# request body with scroll time of 1 minute
request_body = '''{
  "query": {
      "terms":  {
          "ids": %s
      }
  }
  , "size": 100
  , "scroll": "1m"
}''' % (data_list)
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}


# Recursive function to scroll through paginated results
def scroll(scroll_id):
    # Change the request_body to prepare for next scroll api call
    # Make sure to append the include fields to make faster response
    if scroll_id is not None:        
        global request_body
        request_body = '''{"scroll_id": "%s"}''' % (scroll_id)
        
    # make api request
    response = requests.post(url, data=request_body, headers=headers) 
    
    # If rate-limited, wait for n seconds and proceed the same scroll id
    # Since scroll time is 1 minutes, it will give sufficient time to wait and proceed
    
    if response.status_code != 204:
        if response.status_code == requests.codes.too_many_requests:
            time.sleep(8)
            print('TOOO MANY AGGGH')
            scroll(scroll_id)

        # If the response is ok, do something with the response, take the new scroll id and iterate
        
        # leave the function if it returns empty (because then everything is downloaded already)
        else:
            print('next scroll')
            json = response.json()
            if 'scroll_id' in json:
                scroll_id = json['scroll_id']
            else:
                print('No scroll_id found in response.')
                return # Extract the new scroll id from response
            print('Response Status Code:', response.status_code)
            data.append(response.json())
            scroll(scroll_id)
    else: 
        print('empty response')
        
        
# start recursive scrolling
scroll(scroll_id=None)

next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll


In [118]:
patent_ids = []
doc_keys = []
lens_ids = []
dates = []
families = []
codes = []
abstracts = []
claims = []
legal_status = []
publication_type = []

for scroll in data:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                #print(patent)
                patent_ids.append(re.sub(r'_[^_]*$', '', patent['doc_key']).replace('_',''))
                lens_ids.append(patent['lens_id'])
                doc_keys.append(patent['doc_key'])
                dates.append(patent['date_published'])
                families.append(patent['families'])
                if 'classifications_ipcr' in patent['biblio'].keys():  
                    codes.append([x['symbol'] for x in patent['biblio']['classifications_ipcr']['classifications']])
                else: 
                    codes.append(np.nan)
                if 'abstract' in patent.keys(): 
                    abstracts.append(patent['abstract'])
                else: 
                    abstracts.append(np.nan)
                if 'claims' in patent.keys(): 
                    claims.append(patent['claims'])
                else: 
                    claims.append(np.nan)
                
                legal_status.append(patent['legal_status'])
                publication_type.append(patent['publication_type'])
                
                
                
df_patent_data = pd.DataFrame(list(zip(patent_ids,doc_keys,lens_ids,dates, families, codes, abstracts, claims, legal_status, publication_type)), columns = ['patent_id','doc_key','lens_id','date_published', 'families', 'codes', 'abstract', 'claims', 'legal_status', 'publication_type'])

In [150]:
len(patent_ids)

3319

In [134]:
df.head()

,organism,patent_num,patent_url,mod_patent_num
0,Vibrio cholerae,WO2004024931A2,https://patents.google.com/patent/WO2004024931...,WO2004024931A2
1,Vibrio cholerae,JP2010213712A,https://patents.google.com/patent/JP2010213712...,JP2010213712A
2,Vibrio cholerae,JP2012503992A,https://patents.google.com/patent/JP2012503992...,JP2012503992A
3,Vibrio cholerae,JP2012503993A,https://patents.google.com/patent/JP2012503993...,JP2012503993A
4,Vibrio cholerae,JP2011062202A,https://patents.google.com/patent/JP2011062202...,JP2011062202A


In [142]:
#merge with the missing ones
df_patent_data_merge = pd.merge(df_patent_data.rename(columns = {'patent_id': 'patent_num'}), df[['patent_num','organism','patent_url']], how = 'outer', on = 'patent_num')

In [143]:
df_patent_data_merge

,patent_num,doc_key,lens_id,date_published,families,codes,abstract,claims,legal_status,publication_type,organism,patent_url
0,US9410134B2,US_9410134_B2_20160809,000-959-609-762-989,2016-08-09,{'simple_family': {'members': [{'document_id':...,"[C07H21/04, A01K67/027, C12N9/16, C12N9/22, C1...",[{'text': 'The present invention relates to a ...,[{'claims': [{'claim_text': ['1. A nucleic aci...,"{'granted': True, 'grant_date': '2016-08-09', ...",GRANTED_PATENT,Odobenus rosmarus divergens,https://patents.google.com/patent/US9410134B2/...
1,JP2003532698A,JP_2003532698_A_20031105,001-815-591-912-275,2003-11-05,{'simple_family': {'members': [{'document_id':...,"[G01N33/50, A61K31/00, A61K31/12, A61K31/121, ...",NaN,NaN,{'calculation_log': ['Application Filing Date:...,PATENT_APPLICATION,Vibrio cholerae,https://patents.google.com/patent/JP2003532698...
2,US9422558B2,US_9422558_B2_20160823,003-170-265-500-408,2016-08-23,{'simple_family': {'members': [{'document_id':...,"[C12N15/113, A01H3/04, A01N57/16, A01N57/20, A...",[{'text': 'The present invention provides nove...,"[{'claims': [{'claim_text': [""1. A method of p...","{'granted': True, 'grant_date': '2016-08-23', ...",GRANTED_PATENT,Rhizophora mucronata,https://patents.google.com/patent/US9422558B2/...
3,US8420312B2,US_8420312_B2_20130416,004-053-513-540-817,2013-04-16,{'simple_family': {'members': [{'document_id':...,"[C07H21/04, C12Q1/68]",[{'text': 'Described herein is a combination c...,[{'claims': [{'claim_text': ['1. A method to d...,"{'granted': True, 'grant_date': '2013-04-16', ...",GRANTED_PATENT,Trichechus manatus latirostris,https://patents.google.com/patent/US8420312B2/...
4,JP2018511602A,JP_2018511602_A_20180426,004-582-080-858-734,2018-04-26,{'simple_family': {'members': [{'document_id':...,"[C07K19/00, C07K14/435, C07K14/705, C12N5/10, ...",[{'text': '本発明は、細胞シグナル伝達経路並びに他の細胞内及び細胞外タンパク質−タ...,NaN,{'calculation_log': ['Application Filing Date:...,PATENT_APPLICATION,Leptuca pugilator,https://patents.google.com/patent/JP2018511602...
...,...,...,...,...,...,...,...,...,...,...,...,...
4774,WO2014007229A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shewanella fodinae,https://patents.google.com/patent/WO2014007229...
4775,US6562595A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shewanella seohaensis,https://patents.google.com/patent/US6562595A/e...
4776,US6451314A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hypnos monopterygius,https://patents.google.com/patent/US6451314A/e...
4777,US6605709A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pseudoalteromonas sp. s4389,https://patents.google.com/patent/US6605709A/e...


In [122]:
# save it
with open(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_3319.json', 'w') as f:
    json.dump(data, f)

In [149]:
# save it as csv
df_patent_data_merge.to_excel(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_3319.xlsx')


In [146]:
# for exploring the data
for scroll in data:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                print(patent.keys())
                print(patent['biblio']['classifications_ipcr'])
                print(patent['claims'])
                a = pd.json_normalize(patent)
                break
    break

dict_keys(['lens_id', 'jurisdiction', 'doc_number', 'kind', 'date_published', 'doc_key', 'docdb_id', 'lang', 'biblio', 'families', 'legal_status', 'abstract', 'claims', 'description', 'publication_type'])
{'classifications': [{'symbol': 'C07H21/04'}, {'symbol': 'A01K67/027'}, {'symbol': 'C12N9/16'}, {'symbol': 'C12N9/22'}, {'symbol': 'C12N15/62'}, {'symbol': 'C12N15/85'}, {'symbol': 'C12N15/90'}]}
[{'claims': [{'claim_text': ['1. A nucleic acid molecule encoding a fusion protein comprising a DNA-binding domain and (I) a polypeptide having the activity of an endonuclease, wherein the nucleic acid molecule is selected from the group consisting of: (a) a nucleic acid molecule encoding a polypeptide comprising or consisting of the amino acid sequence of SEQ ID NO: 1; (b) a nucleic acid molecule comprising or consisting of the nucleotide sequence of SEQ ID NO: 2; (c) a nucleic acid molecule encoding an endonuclease, the amino acid sequence of said endonuclease is at least 70% identical to t

## Trash 1: Check for flawes

In [45]:
#try with fewer numbers: 
# make list of patentIDs (got to include the double quotes); delete duplicates
data_list2 = json.dumps(list(set(df['mod_patent_num'][:100])))

len(list(set(df['mod_patent_num'])))


4779

In [46]:
data_list2

'["EP2616093A2", "WO2021230804A1", "WO2010020654A2", "WO2008133161A1", "EP2322535A2", "KR1020120079258A", "JP2020000120A", "JP2002503478A", "EP3139965A2", "KR1020200128385A", "JP2014195456A", "EP2543721A1", "EP2096177A2", "WO2004083251A2", "WO0070086A1", "WO0127144A2", "EP2080769A2", "WO2008142034A2", "JP2002085068A", "JP2012183068A", "JP2003532698A", "WO2012107746A1", "WO9935271A2", "WO2008034648A1", "WO2010037111A1", "KR1020130105659A", "WO2004024931A2", "JP2015511601A", "EP2825176A1", "WO03087386A2", "KR1020110063576A", "WO2010046221A1", "WO2010062597A1", "EP2350132A2", "JP2016002034A", "WO2010037119A1", "WO2007011736A2", "JP2012503993A", "JP2013000064A", "JP2013505739A", "WO2007137973A1", "KR1020130027551A", "EP2090662A2", "EP3583219A1", "WO2010037105A1", "WO2012172822A1", "EP2365097A1", "WO2005030186A2", "JP2010246419A", "EP2138585A1", "JP2003526334A", "JP2012503992A", "JP2003526327A", "JP2020011916A", "WO0123604A2", "JP2004508018A", "WO2005090596A2", "WO2018225662A1", "JP20135393

In [47]:
data2 = []
url = 'https://api.lens.org/patent/search'

# include fields

# request body with scroll time of 1 minute
request_body = '''{
  "query": {
      "terms":  {
          "ids": %s
      }
  }
  , "size": 20
  , "scroll": "1m"
}''' % (data_list2)
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}


# Recursive function to scroll through paginated results
def scroll(scroll_id):
    # Change the request_body to prepare for next scroll api call
    # Make sure to append the include fields to make faster response
    if scroll_id is not None:        
        global request_body
        request_body = '''{"scroll_id": "%s"}''' % (scroll_id)
        
    # make api request
    response = requests.post(url, data=request_body, headers=headers) 
    
    # If rate-limited, wait for n seconds and proceed the same scroll id
    # Since scroll time is 1 minutes, it will give sufficient time to wait and proceed
    
    if response.status_code != 204:
        if response.status_code == requests.codes.too_many_requests:
            time.sleep(8)
            print('TOOO MANY AGGGH')
            scroll(scroll_id)

        # If the response is ok, do something with the response, take the new scroll id and iterate
        
        # leave the function if it returns empty (because then everything is downloaded already)
        else:
            print('next scroll')
            json = response.json()
            #print(response.json())
            
            if 'scroll_id' in json:
                scroll_id = json['scroll_id']
            else:
                print('No scroll_id found in response.')
                return # Extract the new scroll id from response
            print('Response Status Code:', response.status_code)
            data2.append(response.json())
            
            scroll(scroll_id)
    else: 
        print('empty response')
        #print('eey finished')
        
        
# start recursive scrolling
scroll(scroll_id=None)

next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
next scroll
Response Status Code: 200
empty response
eey finished


In [48]:
checklist = []
for scroll in data2:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                checklist.append(patent['doc_key'])
                print(patent['doc_key'])
    

JP_2003532698_A_20031105
EP_2350132_A2_20110803
JP_2020000120_A_20200109
JP_2011062202_A_20110331
EP_1486215_A2_20041215
WO_2004024931_A2_20040325
EP_2080769_A2_20090722
WO_2005030186_A2_20050407
WO_2008034648_A1_20080327
JP_2013505739_A_20130221
JP_2002503478_A_20020205
JP_2020011916_A_20200123
EP_3583219_A1_20191225
JP_2010246419_A_20101104
WO_2010037112_A1_20100401
JP_2013539751_A_20131028
WO_2010037105_A1_20100401
EP_2287300_A1_20110223
WO_2008133161_A1_20081106
EP_2365097_A1_20110914
WO_2021230804_A1_20211118
JP_WO2008133161_A1_20100722
WO_2010137130_A1_20101202
JP_2014195456_A_20141016
EP_2322535_A2_20110518
JP_WO2009133882_A1_20110901
WO_2015059690_A1_20150430
WO_2009037279_A1_20090326
JP_WO2010137130_A1_20121112
JP_2006500030_A_20060105
WO_2012172822_A1_20121220
EP_2199304_A1_20100623
WO_2012107746_A1_20120816
WO_2017094793_A1_20170608
WO_2004083251_A2_20040930
JP_WO2017217460_A1_20190404
EP_2825176_A1_20150121
WO_2010046221_A1_20100429
EP_2096177_A2_20090902
WO_2009077611_A2_2

In [49]:
len(checklist)

89

In [53]:
checklist

['JP_2003532698_A_20031105',
 'EP_2350132_A2_20110803',
 'JP_2020000120_A_20200109',
 'JP_2011062202_A_20110331',
 'EP_1486215_A2_20041215',
 'WO_2004024931_A2_20040325',
 'EP_2080769_A2_20090722',
 'WO_2005030186_A2_20050407',
 'WO_2008034648_A1_20080327',
 'JP_2013505739_A_20130221',
 'JP_2002503478_A_20020205',
 'JP_2020011916_A_20200123',
 'EP_3583219_A1_20191225',
 'JP_2010246419_A_20101104',
 'WO_2010037112_A1_20100401',
 'JP_2013539751_A_20131028',
 'WO_2010037105_A1_20100401',
 'EP_2287300_A1_20110223',
 'WO_2008133161_A1_20081106',
 'EP_2365097_A1_20110914',
 'WO_2021230804_A1_20211118',
 'JP_WO2008133161_A1_20100722',
 'WO_2010137130_A1_20101202',
 'JP_2014195456_A_20141016',
 'EP_2322535_A2_20110518',
 'JP_WO2009133882_A1_20110901',
 'WO_2015059690_A1_20150430',
 'WO_2009037279_A1_20090326',
 'JP_WO2010137130_A1_20121112',
 'JP_2006500030_A_20060105',
 'WO_2012172822_A1_20121220',
 'EP_2199304_A1_20100623',
 'WO_2012107746_A1_20120816',
 'WO_2017094793_A1_20170608',
 'WO_200

In [59]:
flawed = []
for patent in list(set(df['mod_patent_num'][:100])):
        #print(patent['doc_key'])
        #print(re.sub(r'_[^_]*$', '', patent['doc_key']))
        if patent not in [re.sub(r'_[^_]*$', '', x).replace('_','') for x in checklist]: 
            flawed.append(patent)
            print(patent)
            

KR1020120079258A
KR1020200128385A
WO0070086A1
WO0127144A2
WO9935271A2
KR1020130105659A
WO03087386A2
KR1020110063576A
WO2007137973A1
KR1020130027551A
WO0123604A2
KR1020180083904A
WO0185664A2
WO0208459A2
WO0168829A2
KR1020140140580A
KR1020130105649A
WO2007110314A1


In [60]:
df.head()

,organism,patent_num,patent_url,mod_patent_num
0,Vibrio cholerae,WO2004024931A2,https://patents.google.com/patent/WO2004024931...,WO2004024931A2
1,Vibrio cholerae,JP2010213712A,https://patents.google.com/patent/JP2010213712...,JP2010213712A
2,Vibrio cholerae,JP2012503992A,https://patents.google.com/patent/JP2012503992...,JP2012503992A
3,Vibrio cholerae,JP2012503993A,https://patents.google.com/patent/JP2012503993...,JP2012503993A
4,Vibrio cholerae,JP2011062202A,https://patents.google.com/patent/JP2011062202...,JP2011062202A


In [63]:
df['patent_url'][df['patent_num'].isin(flawed)]

30    https://patents.google.com/patent/KR1020140140...
39    https://patents.google.com/patent/KR1020180083...
42    https://patents.google.com/patent/KR1020120079...
43    https://patents.google.com/patent/KR1020110063...
44    https://patents.google.com/patent/KR1020130027...
45    https://patents.google.com/patent/KR1020130105...
46    https://patents.google.com/patent/KR1020130105...
53    https://patents.google.com/patent/KR1020200128...
63    https://patents.google.com/patent/WO9935271A2/...
64    https://patents.google.com/patent/WO0070086A1/...
65    https://patents.google.com/patent/WO0123604A2/...
66    https://patents.google.com/patent/WO0127144A2/...
67    https://patents.google.com/patent/WO0168829A2/...
68    https://patents.google.com/patent/WO0185664A2/...
69    https://patents.google.com/patent/WO0208459A2/...
76    https://patents.google.com/patent/WO03087386A2...
Name: patent_url, dtype: object

## Trash 2: Get granted patents and map ICP classification labels on codes

In [31]:
with open(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_EP_test.json') as json_string:
    data = json.load(json_string)

In [23]:
for scroll in data:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                print(patent.keys())
                #print(patent['claims'])
                a = pd.json_normalize(patent)
                break
    break

dict_keys(['lens_id', 'jurisdiction', 'doc_number', 'kind', 'date_published', 'doc_key', 'docdb_id', 'lang', 'biblio', 'families', 'legal_status', 'abstract', 'claims', 'description', 'publication_type'])


In [7]:
a

,lens_id,jurisdiction,doc_number,kind,date_published,doc_key,docdb_id,lang,abstract,claims,...,families.simple_family.size,families.extended_family.members,families.extended_family.size,legal_status.granted,legal_status.grant_date,legal_status.anticipated_term_date,legal_status.calculation_log,legal_status.patent_status,description.text,description.lang
0,001-712-126-350-412,EP,1911837,A2,2008-04-16,EP_1911837_A2_20080416,289466762,en,[{'text': 'The invention provides isolated nuc...,[{'claims': [{'claim_text': ['An isolated nucl...,...,56,"[{'document_id': {'jurisdiction': 'JP', 'doc_n...",59,True,2011-05-25,2021-09-28,"[Application Filing Date: 2001-09-28, Earliest...",EXPIRED,Related Applications This application claims p...,en


In [8]:
patent

{'lens_id': '001-712-126-350-412',
 'jurisdiction': 'EP',
 'doc_number': '1911837',
 'kind': 'A2',
 'date_published': '2008-04-16',
 'doc_key': 'EP_1911837_A2_20080416',
 'docdb_id': 289466762,
 'lang': 'en',
 'biblio': {'publication_reference': {'jurisdiction': 'EP',
   'doc_number': '1911837',
   'kind': 'A2',
   'date': '2008-04-16'},
  'application_reference': {'jurisdiction': 'EP',
   'doc_number': '07019867',
   'kind': 'A',
   'date': '2001-09-28'},
  'priority_claims': {'claims': [{'jurisdiction': 'EP',
     'doc_number': '01985723',
     'kind': 'A',
     'date': '2001-09-28',
     'sequence': 1},
    {'jurisdiction': 'US',
     'doc_number': '23630300',
     'kind': 'P',
     'date': '2000-09-28',
     'sequence': 2},
    {'jurisdiction': 'US',
     'doc_number': '29756201',
     'kind': 'P',
     'date': '2001-06-12',
     'sequence': 3}],
   'earliest_claim': {'date': '2000-09-28'}},
  'invention_title': [{'text': 'FAD5-2, Mitglied der Desaturase-Familie und Verwendungen da

In [18]:
# filter those that have granted patents
nr = 1
for scroll in data:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                print(nr)
                if patent['publication_type'] == 'PATENT_APPLICATION': 
                    
                    print(patent['publication_type'])
                    #print(patent['legal_status'])
                    #print(patent['legal_status']['granted'])
                else: 
                    print(patent['publication_type'])
                    break
                #print(patent.keys())
                #print(patent['claims'])
                #a = pd.json_normalize(patent)
                #break
                nr = nr+1
                
    

1
PATENT_APPLICATION
2
PATENT_APPLICATION
3
PATENT_APPLICATION
4
PATENT_APPLICATION
5
PATENT_APPLICATION
6
PATENT_APPLICATION
7
PATENT_APPLICATION
8
PATENT_APPLICATION
9
PATENT_APPLICATION
10
PATENT_APPLICATION
11
PATENT_APPLICATION
12
PATENT_APPLICATION
13
PATENT_APPLICATION
14
PATENT_APPLICATION
15
PATENT_APPLICATION
16
PATENT_APPLICATION
17
PATENT_APPLICATION
18
PATENT_APPLICATION
19
PATENT_APPLICATION
20
PATENT_APPLICATION
21
PATENT_APPLICATION
22
PATENT_APPLICATION
23
PATENT_APPLICATION
24
PATENT_APPLICATION
25
PATENT_APPLICATION
26
PATENT_APPLICATION
27
PATENT_APPLICATION
28
PATENT_APPLICATION
29
PATENT_APPLICATION
30
PATENT_APPLICATION
31
PATENT_APPLICATION
32
PATENT_APPLICATION
33
PATENT_APPLICATION
34
PATENT_APPLICATION
35
PATENT_APPLICATION
36
PATENT_APPLICATION
37
PATENT_APPLICATION
38
PATENT_APPLICATION
39
PATENT_APPLICATION
40
PATENT_APPLICATION
41
PATENT_APPLICATION
42
PATENT_APPLICATION
43
PATENT_APPLICATION
44
PATENT_APPLICATION
45
PATENT_APPLICATION
46
PATENT_APPLICATI

In [ ]:
# all of these are patent_applications

In [19]:
# filter those that have granted patents
nr = 1
data_active = {}

for scroll in data:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                if patent['legal_status']['patent_status'] == 'ACTIVE':
                    print(nr)
                    data_active[patent['lens_id']] = patent
                
                    print(patent['legal_status'])
                    
                    #print(patent['legal_status']['patent_status'])
                
                #print(patent.keys())
                #print(patent['claims'])
                #a = pd.json_normalize(patent)
                #break
                    nr = nr+1
                

1
{'granted': True, 'grant_date': '2016-08-09', 'anticipated_term_date': '2032-06-06', 'calculation_log': ['Application Filing Date: 2012-06-06', 'Earliest Filing Date: 2012-06-06 priority to EP2012060711W', 'Granted Date: 2016-08-09', 'Anticipated Termination Date: 2032-06-06'], 'patent_status': 'ACTIVE'}
2
{'granted': True, 'grant_date': '2016-08-23', 'anticipated_term_date': '2034-12-05', 'calculation_log': ['Application Filing Date: 2012-09-13', 'Granted Date: 2016-08-23', 'Applied 813 days term extension.', 'Anticipated Termination Date: 2034-12-05'], 'patent_status': 'ACTIVE'}
3
{'granted': True, 'grant_date': '2015-08-04', 'anticipated_term_date': '2032-11-21', 'calculation_log': ['Application Filing Date: 2012-11-21', 'Granted Date: 2015-08-04', 'Anticipated Termination Date: 2032-11-21'], 'patent_status': 'ACTIVE'}
4
{'granted': True, 'grant_date': '2017-06-06', 'anticipated_term_date': '2033-10-11', 'calculation_log': ['Application Filing Date: 2013-09-24', 'Earliest Filing D

In [ ]:
# 1107 out of 457 are active. not sure what exactly this is based on (could some of these be renewed versions? does it take the entire family into accoutn? )

In [24]:
nr = 1
for k,patent in data_active.items(): 
    
    if patent['legal_status']['granted'] != True:
        print(nr)
                
        print(patent['legal_status'])
                    
                    #print(patent['legal_status']['patent_status'])
                
                #print(patent.keys())
                #print(patent['claims'])
                #a = pd.json_normalize(patent)
                #break
        nr = nr+1

In [ ]:
# does granted and active mean exactly the same thing? a bit strange.. 

In [ ]:
# I guess the point would be to find these applications on the lens but then move on to the actual granted patent for the actual data work

## 3. Check ICPR classifications

In [25]:
classifications = []
for k,patent in data_active.items(): 
    for symbol in patent['biblio']['classifications_ipcr']['classifications']: 
        classifications.append(symbol['symbol'])
    
    

In [26]:
len(set(classifications))

1137

In [27]:
count = Counter(classifications)
count.most_common()

[('C12N15/09', 240),
 ('C12N1/21', 147),
 ('C12N15/82', 139),
 ('C12N5/10', 131),
 ('C12Q1/68', 121),
 ('C12N1/19', 107),
 ('C12N9/02', 103),
 ('C12N9/10', 102),
 ('C12N1/15', 93),
 ('A61K38/00', 89),
 ('C12P7/64', 88),
 ('A61K48/00', 85),
 ('C07K14/47', 81),
 ('C12N9/00', 74),
 ('C07H21/04', 73),
 ('C12N15/00', 62),
 ('A61K39/00', 60),
 ('C12N15/113', 59),
 ('C12P21/02', 57),
 ('C07K14/435', 55),
 ('A01H5/00', 53),
 ('A61K38/17', 51),
 ('C12N9/12', 50),
 ('C12N15/11', 49),
 ('C12N15/52', 49),
 ('C12N9/88', 48),
 ('C12N15/63', 47),
 ('C12N1/20', 46),
 ('A61K39/395', 45),
 ('G01N33/53', 42),
 ('G01N33/68', 42),
 ('C12N15/85', 38),
 ('C12N9/04', 37),
 ('C07K19/00', 37),
 ('C12N9/16', 36),
 ('C07K16/18', 36),
 ('C07K14/705', 35),
 ('A61K45/06', 35),
 ('C07K16/28', 34),
 ('G01N33/50', 32),
 ('G01N33/574', 32),
 ('C07K16/00', 32),
 ('C07K14/415', 32),
 ('C07K14/00', 31),
 ('C07K14/195', 30),
 ('A01K67/027', 29),
 ('A61K38/18', 29),
 ('A61P43/00', 29),
 ('C12P21/00', 26),
 ('C12Q1/02', 25),


In [29]:
classification_descriptions = []
for term in classifications: 
    my_ipc = Ipc(term[:4])
    classification_descriptions.append(my_ipc.classe[1])

In [30]:
count = Counter(classification_descriptions)
count.most_common()

[('BIOCHEMISTRY; BEER; SPIRITS; WINE; VINEGAR; MICROBIOLOGY; ENZYMOLOGY; MUTATION OR GENETIC ENGINEERING',
  3212),
 ('MEDICAL OR VETERINARY SCIENCE; HYGIENE', 1406),
 ('ORGANIC CHEMISTRY', 980),
 ('MEASURING; TESTING', 289),
 ('AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING', 250),
 ('FOODS OR FOODSTUFFS; THEIR TREATMENT, NOT COVERED BY OTHER CLASSES', 147),
 ('ANIMAL OR VEGETABLE OILS, FATS, FATTY SUBSTANCES OR WAXES; FATTY ACIDS THEREFROM; DETERGENTS; CANDLES',
  69),
 ('INFORMATION AND COMMUNICATION TECHNOLOGY [ICT] SPECIALLY ADAPTED FOR SPECIFIC APPLICATION FIELDS',
  37),
 ('PETROLEUM, GAS OR COKE INDUSTRIES; TECHNICAL GASES CONTAINING CARBON MONOXIDE; FUELS; LUBRICANTS; PEAT',
  28),
 ('TREATMENT OF TEXTILES OR THE LIKE; LAUNDERING; FLEXIBLE MATERIALS NOT OTHERWISE PROVIDED FOR',
  16),
 ('COMBINATORIAL TECHNOLOGY', 14),
 ('COMPUTING; CALCULATING; COUNTING', 10),
 ('PHYSICAL OR CHEMICAL PROCESSES OR APPARATUS IN GENERAL', 9),
 ('DYES; PAINTS; POLISHES; NATUR